In [4]:
import datetime
import gpm
import pyvista as pv
from gpm.visualization.plot_3d import (
    create_pyvista_2d_surface, 
    create_pyvista_3d_volume,
)
from gpm.utils.geospatial import extend_geographic_extent

In [2]:
# micromamba install pyvista trame trame-vuetify trame-vtk 

In [ ]:
# Define plot settings
background_color = "#2e2a2b"

# Load GPM DPR overpass

In [27]:
# Specify the time period you are interested in 
start_time = datetime.datetime.strptime("2021/08/29 15:10:00", "%Y/%m/%d %H:%M:%S")
end_time = datetime.datetime.strptime("2021/08/29 15:18:00", "%Y/%m/%d %H:%M:%S")
# Specify the product and product type 
product = "2A-DPR"    # 2A-PR
product_type = "RS"   
# Specify the version
version = 7
# Specify variables 
variables = ["zFactorFinalNearSurface", "zFactorFinal"]

# Load the 2A-DPR dataset
# - If scan_mode is not specified, it automatically load one! 
ds = gpm.open_dataset(
    product=product,
    product_type=product_type,
    version=version,
    variables=variables,
    start_time=start_time,
    end_time=end_time,
)
ds



'scan_mode' has not been specified. Default to FS.


<xarray.Dataset> Size: 72MB
Dimensions:                  (cross_track: 49, along_track: 687, range: 176,
                              radar_frequency: 2)
Coordinates: (12/14)
    height                   (cross_track, along_track, range) float32 24MB dask.array<chunksize=(49, 687, 176), meta=np.ndarray>
    SCorientation            (along_track) float32 3kB dask.array<chunksize=(687,), meta=np.ndarray>
    dataQuality              (along_track, radar_frequency) float32 5kB dask.array<chunksize=(687, 2), meta=np.ndarray>
    lon                      (cross_track, along_track) float32 135kB -94.77 ...
    lat                      (cross_track, along_track) float32 135kB 17.25 ....
    time                     (along_track) datetime64[ns] 5kB 2021-08-29T15:1...
    ...                       ...
    gpm_cross_track_id       (cross_track) int64 392B ...
    gpm_along_track_id       (along_track) int64 5kB ...
  * range                    (range) int64 1kB 1 2 3 4 5 ... 172 173 174 175 176
    gpm_range_id             (range) int64 1kB ...
  * radar_frequency          (radar_frequency) <U2 16B 'Ku' 'Ka'
    crsWGS84                 int64 8B 0
Dimensions without coordinates: cross_track, along_track
Data variables:
    zFactorFinal             (cross_track, along_track, range, radar_frequency) float32 47MB dask.array<chunksize=(49, 687, 176, 2), meta=np.ndarray>
    zFactorFinalNearSurface  (cross_track, along_track, radar_frequency) float32 269kB dask.array<chunksize=(49, 687, 2), meta=np.ndarray>
Attributes: (12/23)
    FileName:              2A.GPM.DPR.V9-20211125.20210829-S144151-E161423.04...
    EphemerisFileName:     
    AttitudeFileName:      
    TotalQualityCode:      Good
    DielectricFactorKa:    0.8989
    DielectricFactorKu:    0.9255
    ...                    ...
    DataFormatVersion:     7h
    MetadataVersion:       7h
    ProcessingMode:        STD
    ScanMode:              FS
    history:               Created by ghiggi/gpm_api software on 2025-02-28 1...
    gpm_api_product:       2A-DPR

In [28]:
# Define bounding box of interest
extent = [-95, -84, 26, 33]
bbox_extent = extend_geographic_extent(extent, padding=0.5)  

# Crop dataset
ds = ds.gpm.crop(bbox_extent).compute()
ds


<xarray.Dataset> Size: 24MB
Dimensions:                  (cross_track: 49, along_track: 229, range: 176,
                              radar_frequency: 2)
Coordinates: (12/14)
    height                   (cross_track, along_track, range) float32 8MB 2....
    SCorientation            (along_track) float32 916B 180.0 180.0 ... 180.0
    dataQuality              (along_track, radar_frequency) float32 2kB 0.0 ....
    lon                      (cross_track, along_track) float32 45kB -91.31 ....
    lat                      (cross_track, along_track) float32 45kB 24.46 .....
    time                     (along_track) datetime64[ns] 2kB 2021-08-29T15:1...
    ...                       ...
    gpm_cross_track_id       (cross_track) int64 392B 0 1 2 3 4 ... 45 46 47 48
    gpm_along_track_id       (along_track) int64 2kB 2591 2592 ... 2818 2819
  * range                    (range) int64 1kB 1 2 3 4 5 ... 172 173 174 175 176
    gpm_range_id             (range) int64 1kB 0 1 2 3 4 ... 171 172 173 174 175
  * radar_frequency          (radar_frequency) <U2 16B 'Ku' 'Ka'
    crsWGS84                 int64 8B 0
Dimensions without coordinates: cross_track, along_track
Data variables:
    zFactorFinal             (cross_track, along_track, range, radar_frequency) float32 16MB ...
    zFactorFinalNearSurface  (cross_track, along_track, radar_frequency) float32 90kB ...
Attributes: (12/23)
    FileName:              2A.GPM.DPR.V9-20211125.20210829-S144151-E161423.04...
    EphemerisFileName:     
    AttitudeFileName:      
    TotalQualityCode:      Good
    DielectricFactorKa:    0.8989
    DielectricFactorKu:    0.9255
    ...                    ...
    DataFormatVersion:     7h
    MetadataVersion:       7h
    ProcessingMode:        STD
    ScanMode:              FS
    history:               Created by ghiggi/gpm_api software on 2025-02-28 1...
    gpm_api_product:       2A-DPR

In [33]:
isel_dict = {"cross_track": slice(0,49), "along_track": slice(40, 180)}
ds_patch = ds.isel(isel_dict)

# Create PyVista objects

In [34]:
# Retrieve pyvista surface and volume data objects
radar_frequency="Ku"
da_2d = ds_patch["zFactorFinalNearSurface"].sel(radar_frequency=radar_frequency)
surf = create_pyvista_2d_surface(da_2d, spacing=(1, 1, 1), origin=(0, 0, -3))

da_3d = ds_patch["zFactorFinal"].sel(radar_frequency=radar_frequency)
vol = create_pyvista_3d_volume(da_3d, spacing=(1, 1, 0.25), origin=(0, 0, 0))

In [35]:
# Define figure settings 
cmap="Spectral_r"
clim=[10, 50]

# 2D Surface Map Visualization

In [36]:
# Display
pl = pv.Plotter()
pl.background_color = background_color
pl.add_mesh(surf, opacity=1, cmap=cmap, clim=clim, show_scalar_bar=False)
pl.show()

Widget(value='<iframe src="http://localhost:38033/index.html?ui=P_0x7f291f041110_12&reconnect=auto" class="pyv…

# 3D Isosurfaces Visualization

In [37]:
from gpm.visualization.plot_3d import add_3d_isosurfaces

isovalues=[30, 40, 50]
opacities=[0.3, 0.5, 1]
method="contour"

# Display figure 
pl = pv.Plotter([1024*2, 1024*3], image_scale=3)
pl.background_color = background_color
pl.add_mesh(surf, opacity=1, cmap=cmap, clim=clim, show_scalar_bar=False, style="surface")
add_3d_isosurfaces(vol=vol, pl=pl, 
                   isovalues=isovalues,
                   opacities=opacities,
                   method=method,
                   cmap=cmap,
                   clim=clim,
                   show_scalar_bar=False,
                   add_sliders=True,
                  )
pl.show()

Widget(value='<iframe src="http://localhost:38033/index.html?ui=P_0x7f291f0ecc50_13&reconnect=auto" class="pyv…

In [38]:
# Display figure 
pl = pv.Plotter([1024*2, 1024*3], image_scale=3)
pl.background_color = background_color
pl.add_mesh(surf, opacity=1, cmap=cmap, clim=clim, show_scalar_bar=False, style="surface")
add_3d_isosurfaces(vol=vol, pl=pl, 
                   isovalues=isovalues,
                   opacities=opacities,
                   method=method,
                   cmap=cmap,
                   clim=clim,
                   show_scalar_bar=False,
                   add_sliders=False,
                  )
pl.show()

Widget(value='<iframe src="http://localhost:38033/index.html?ui=P_0x7f291ef9a6d0_14&reconnect=auto" class="pyv…

# 3D Isosurface Interactive Slider

In [ ]:
import pyvista as pv
from gpm.visualization.plot_3d import add_3d_isosurface_slider

# Display figure 
pl = pv.Plotter([1024*2, 1024*3], image_scale=3)
pl.background_color = background_color
pl.add_mesh(surf, opacity=1, cmap=cmap, clim=clim, show_scalar_bar=False, style="surface")
add_3d_isosurface_slider(vol=vol, pl=pl, isovalue=30, opacity=1, cmap=cmap, clim=clim, show_scalar_bar=False, style="surface")
pl.show()

Widget(value='<iframe src="http://localhost:38033/index.html?ui=P_0x7f29ec53a3d0_3&reconnect=auto" class="pyvi…

# 3D Orthogonal Slices Visualization

In [ ]:
from gpm.visualization.plot_3d import add_3d_orthogonal_slices

pl = pv.Plotter([1024*3, 768*3], image_scale=3)
pl.background_color = background_color
add_3d_orthogonal_slices(vol=vol, pl=pl, x=22, y=20, z=1, add_sliders=True, clim=clim, cmap=cmap, show_scalar_bar=False)
pl.add_scalar_bar(vertical=False, title="Reflectivity [dBZ]", color="white", height=0.2, width=.2, position_x=0.7, position_y=0.8)
pl.show()

Widget(value='<iframe src="http://localhost:38033/index.html?ui=P_0x7f29c8da6ad0_4&reconnect=auto" class="pyvi…

# 3D Volume Single Plane Slicer 

In [ ]:
pl = pv.Plotter()
pl.background_color = background_color
pl.add_mesh(surf, clim=clim, cmap=cmap, show_scalar_bar=False)
pl.add_volume_clip_plane(vol, normal='-x', cmap=cmap, clim=clim)
pl.show()

Widget(value='<iframe src="http://localhost:38033/index.html?ui=P_0x7f29c8d8c350_5&reconnect=auto" class="pyvi…

# 3D Volume Vertical Slicer

In [ ]:
pl = pv.Plotter()
pl.background_color = background_color
pl.add_volume_clip_plane(vol, normal='-z', normal_rotation=False, cmap=cmap, clim=clim)
pl.show()

Widget(value='<iframe src="http://localhost:38033/index.html?ui=P_0x7f29bc3ff0d0_6&reconnect=auto" class="pyvi…

# 3D Volume Orthogonal Slicer

In [39]:
pl = pv.Plotter()
pl.background_color = background_color
pl.add_mesh(surf, clim=clim, cmap=cmap, show_scalar_bar=False)
pl.add_volume_clip_plane(
    vol,
    normal='-x',
    interaction_event='always',
    normal_rotation=False,
    cmap=cmap, clim=clim,
    show_scalar_bar=False,
)
pl.add_volume_clip_plane(
    vol,
    normal='-y',
    interaction_event='always',
    normal_rotation=False,
    cmap=cmap, clim=clim,
    show_scalar_bar=False,
)
pl.show()

Widget(value='<iframe src="http://localhost:38033/index.html?ui=P_0x7f2949bc4f10_15&reconnect=auto" class="pyv…

In [18]:
background_color = "#FFFFFF"